# Model 2 Version 1 - Clustering Using Average Song and Playlist Features Values.
Our first attempt at model 2 uses the assumption that the data is Gaussian and perform KNN with the feature values of each song.

This code block imports all necessary libraries and fetches the average feature values given a title for the playlist.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3
import json
"""
#Connects to the title dataset
title_conn = sqlite3.Connection("../../data/title/title_dataset.db")
titleCur = title_conn.cursor()
# Selects database entry titles with title run it
data = titleCur.execute("SELECT * FROM TITLES WHERE title = 'run it'")
listOfResults = data.fetchall()
averageFeatures = np.zeros(24) #Datatype that represents the average feature
#Calculates the sum of all results
for result in listOfResults:
    resultList = list(result) #Converts tuple into list
    resultList.pop(0) #Removes title from list
    resultArray = np.asarray(resultList) #converts list to numpy array
    averageFeatures = np.add(resultArray,averageFeatures) #Sums all features
averageFeatures = np.divide(averageFeatures, len(listOfResults)) #Calulates average feature value
"""
FEATURE_NAMES = [
    "acousticness",
    "danceability",
    "duration_ms",
    "energy",
    "instrumentalness",
    "liveness",
    "loudness",
    "mode",
    "speechiness",
    "tempo",
    "time_signature",
    "valence"
]
def process_playlist(playlist: dict, song_conn: sqlite3.Connection):
    """Process a single playlist from its JSON dictionary."""
    ids = [id_from_uri(track['track_uri']) for track in playlist['tracks']]

    # Queries for all the given songs. 
    # The interpolated bit expands to (?,?,?,?,...,?) for as many ids as we have,
    # so this is still a prepared statement and there is no SQL injection vulnerability.
    songs = pd.read_sql(
        "SELECT * FROM songs WHERE id IN ({0})".format(','.join('?' for _ in ids)),
        song_conn,
        params=ids,
    )

    means = songs[FEATURE_NAMES].mean(axis=0).add_suffix("_mean")
    sds = songs[FEATURE_NAMES].std(axis=0).add_suffix("_sd")

    playlist_df = pd.concat((means, sds), axis=0)
    return playlist_df
def id_from_uri(uri: str):
    """Helper method to get the ID from a URI string like so:
    URI: 'spotify:artist:012345...'
    ID: '012345...'
    """
    return uri.split(':')[2]
def get_songs_title_from_playlist_slice(slice_json, index):
    """Get the list of songs (IDs) and the title of a playlist
    given the slice JSON its from and the playlist's index.
    """
    playlist_json = slice_json['playlists'][index]
    return [id_from_uri(track['track_uri']) for track in playlist_json['tracks']], \
           playlist_json['name']

with open('../../data/playlist/mpd.slice.0-999.json') as testSlice:
    slice_json = json.load(testSlice)

listOfSongIDs = get_songs_title_from_playlist_slice(slice_json, 0) #Gets list of Song IDs from the playlist
song_conn = sqlite3.Connection("../../data/song/song_dataset.db")
overall_df = pd.DataFrame()
listOfPlaylists = slice_json['playlists']
playlist_row = process_playlist(listOfPlaylists[0], song_conn)
selected_columns = ['acousticness_mean', 'acousticness_sd', 'danceability_mean', 'danceability_sd','duration_ms_mean','duration_ms_sd', 'energy_mean', 'energy_sd', 'instrumentalness_mean', 'instrumentalness_sd', 'liveness_mean', 'liveness_sd', 'loudness_mean', 'loudness_sd', 'mode_mean', 'mode_sd', 'speechiness_mean', 'speechiness_sd', 'tempo_mean','tempo_sd', 'time_signature_mean', 'time_signature_sd', 'valence_mean', 'valence_sd']
averageFeatures = (playlist_row[selected_columns]).values
print(averageFeatures)

# Load a slice of the playlists
# Get the songs and title of one of them by indexing into it with the helper function
# Query the songs database to get the features of the songs
# Compute mean, std. dev. again


[ 8.53018412e-02  1.20975877e-01  6.68784314e-01  1.44455019e-01
  2.22076627e+05  2.57998910e+04  7.77960784e-01  1.28349049e-01
  6.87605098e-04  3.61208866e-03  1.87264706e-01  1.47960230e-01
 -4.90845098e+00  1.33224483e+00  7.05882353e-01  4.60178993e-01
  1.04466667e-01  8.72788902e-02  1.20593255e+02  3.19770345e+01
  4.00000000e+00  0.00000000e+00  6.45470588e-01  1.94765434e-01]


The following block of code retrieves the songs in the song dataset that are within the average and standard deviation values for each feature determined in the previous coding block. In other words it is determining the cluster of songs that are relevant to our given playlist.

In [2]:
#Connects to the Song dataset
song_conn = sqlite3.Connection("../../data/song/song_dataset.db")
songCur = song_conn.cursor()
query = "SELECT * FROM songs WHERE (acousticness BETWEEN ? AND ?) AND (danceability BETWEEN ? AND ?) AND (energy BETWEEN ? AND ?) AND (instrumentalness BETWEEN ? AND ?) AND (liveness BETWEEN ? AND ?) AND (loudness BETWEEN ? AND ?) AND (mode BETWEEN ? AND ?) AND (speechiness BETWEEN ? AND ?) AND (tempo BETWEEN ? AND ?) AND (time_signature BETWEEN ? AND ?) AND (valence BETWEEN ? AND ?) "
#Determines accousticness bounds
accousticNessLower = averageFeatures[0] - averageFeatures[1]
accousticNessUpper = averageFeatures[0] + averageFeatures[1]
#Determines danceability bounds
danceLower = averageFeatures[2] - averageFeatures[3]
danceUpper = averageFeatures[2] + averageFeatures[3]
#Determines energy bounds
energyLower = averageFeatures[6] - averageFeatures[7]
energyUpper = averageFeatures[6] + averageFeatures[7]
#Determines instrumentalness bounds
instrumentalnessLower = averageFeatures[8] - averageFeatures[9]
instrumentalnessUpper = averageFeatures[8] + averageFeatures[9]
#Determines liveness bounds
livenessLower = averageFeatures[10] - averageFeatures[11]
livenessUpper = averageFeatures[10] + averageFeatures[11]
#Determines loudness bounds
loudnessLower = averageFeatures[12] - averageFeatures[13]
loudnessUpper = averageFeatures[12] + averageFeatures[13]
#Determines mode bounds
modeLower = averageFeatures[14] - averageFeatures[15]
modeUpper = averageFeatures[14] + averageFeatures[15]
#Determines speechiness bounds
speechinessLower = averageFeatures[16] - averageFeatures[17]
speechinessUpper = averageFeatures[16] + averageFeatures[17]
#Determines tempo bounds
tempoLower = averageFeatures[18] - averageFeatures[19]
tempoUpper = averageFeatures[18] + averageFeatures[19]
#Determines time signature bounds
timeSigLower = averageFeatures[20] - averageFeatures[21]
timeSigUpper = averageFeatures[20] + averageFeatures[21]
#Determines valence signature bounds
valenceLower = averageFeatures[22] - averageFeatures[23]
valenceUpper = averageFeatures[22] + averageFeatures[23]
#Executes the SQL query to retrieve data that are within bounds
desiredSong = songCur.execute(query, (accousticNessLower,accousticNessUpper, danceLower, danceUpper,energyLower, energyUpper, instrumentalnessLower, instrumentalnessUpper, livenessLower, livenessUpper, loudnessLower, loudnessUpper, modeLower, modeUpper, speechinessLower, speechinessUpper, tempoLower, tempoUpper, timeSigLower, timeSigUpper, valenceLower, valenceUpper))
data = pd.read_sql(query, song_conn, params=(accousticNessLower,accousticNessUpper, danceLower, danceUpper,energyLower, energyUpper, instrumentalnessLower, instrumentalnessUpper, livenessLower, livenessUpper, loudnessLower, loudnessUpper, modeLower, modeUpper, speechinessLower, speechinessUpper, tempoLower, tempoUpper, timeSigLower, timeSigUpper, valenceLower, valenceUpper))

Using the cluster that we have determined to be appropriate, we analyze each song in the cluster and determine a ranked list of songs sorting from closest song to farthest song.

In [3]:
#Extracts only the columns that contain the feature values of interest
listOfColumns = ['acousticness','danceability','energy', 'instrumentalness', 'liveness', 'loudness', 'mode','speechiness', 'tempo', 'time_signature', 'valence']
featureMatrix = data.loc[:, listOfColumns]
#Converts the pandas datafram into a numpy array.
featureMatrix = featureMatrix.to_numpy() 
#Contains the mean values for all features
meanMatrix = np.array([averageFeatures[0], averageFeatures[2], averageFeatures[6], averageFeatures[8], averageFeatures[10], averageFeatures[12], averageFeatures[14], averageFeatures[16], averageFeatures[18], averageFeatures[20], averageFeatures[22]])
#Contains the std deviation values for all features.
stdDevMatrix = np.array([averageFeatures[1], averageFeatures[3], averageFeatures[7], averageFeatures[9], averageFeatures[11], averageFeatures[13], averageFeatures[15], averageFeatures[17], averageFeatures[19], averageFeatures[21], averageFeatures[23]])
stdDevMatrix = stdDevMatrix.tolist()
for i in range(len(stdDevMatrix)):
    if stdDevMatrix[i] == 0:
        stdDevMatrix[i] = 1e-10
stdDevMatrix = np.array(stdDevMatrix)
#Calculated distances from each song to center
print(stdDevMatrix)
distanceMatrix = np.subtract(featureMatrix, meanMatrix) # d - m
distanceMatrix = np.divide(distanceMatrix, stdDevMatrix) # (d-m) / std deviation
distanceMatrix = np.power(distanceMatrix, 2) #Squares the previous result
distanceMatrix = np.sum(distanceMatrix, axis=1) #Sums over the entire row
distanceMatrix = np.sqrt(distanceMatrix) #Square roots the result
songIDList = data.loc[:, ['id']]
songIDList = list(songIDList.values.flatten().tolist())
rankedList = list(zip(songIDList, distanceMatrix))
rankedList = sorted(rankedList, key=lambda distance: distance[1])
print(rankedList[0:10])

[1.20975877e-01 1.44455019e-01 1.28349049e-01 3.61208866e-03
 1.47960230e-01 1.33224483e+00 4.60178993e-01 8.72788902e-02
 3.19770345e+01 1.00000000e-10 1.94765434e-01]
[('2mqyzKOnCL4g2Tavpi7UlW', 0.9061428688601596), ('7f5XDgmeSqCNgpTAOSVRUh', 0.9147244493385711), ('3nPIWr2WnEdbc9P8hrkLbj', 0.9658335833143283), ('3tMyxF6Erg2AQPLTY9pMxx', 0.9697834988222916), ('6eolFkFTZmdai6b47dxKtJ', 0.9732246065890361), ('35BXOeMdVF2TnuEnfnJecW', 0.9796994670561486), ('4YMJTBzfR8dhdPE3wrguw6', 0.9899436554501699), ('40bxDmUIpgXDff9g1lPn2T', 0.9917430945648118), ('0KJmsQNMPf2rG5E1UbroBs', 0.9924239706111982), ('2KPL20JCgAADW3VTGlfmJQ', 0.9998346920058268)]
